In [1]:
#
# Recurrent Neural Network MNIST
#
# @author becxer
# @email becxer87@gmail.com
# @reference https://github.com/sjchoi86/Tensorflow-101
#

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline  

print ("packages are loaded")

packages are loaded


In [2]:
# Get MNIST data set from tensorflow
from tensorflow.examples.tutorials.mnist import input_data
mnist_set = input_data.read_data_sets('images/MNIST', one_hot = True)

train_img = mnist_set.train.images
train_label = mnist_set.train.labels
test_img = mnist_set.test.images
test_label = mnist_set.test.labels

Extracting images/MNIST/train-images-idx3-ubyte.gz
Extracting images/MNIST/train-labels-idx1-ubyte.gz
Extracting images/MNIST/t10k-images-idx3-ubyte.gz
Extracting images/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
# Options for training
learning_rate = 0.001
training_epochs = 50
batch_size = 100
display_step = 10

# Define dimensions
input_dimen = 28
hidden_dimen = 128
output_dimen = 10

rstep = 28 # 28 * 28 to 28 steps for 28

In [4]:
# Build Graph of Recurrent Neural Network

WS = {
    'hidden' : tf.Variable(tf.random_normal([input_dimen, hidden_dimen])),
    'out' : tf.Variable(tf.random_normal([hidden_dimen, output_dimen]))
}

BS = {
    'hidden' : tf.Variable(tf.random_normal([hidden_dimen])),
    'out' : tf.Variable(tf.random_normal([output_dimen]))
}

def SimpleRNN(init_cell_state, rstep, X_, W_, B_):
    # Change input form 
    # [batch size, height(rstep), weight(x)]  -> [rstep, batch size, x]
    X_ = tf.transpose(X_, [1, 0, 2])
    X_ = tf.reshape(X_,[-1, input_dimen])
    H_ = tf.add(tf.matmul(X_, W_['hidden']), B_['hidden'])
    H_split = tf.split(0, rstep, H_)
    
    #struct lstm RNN cell
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(hidden_dimen, forget_bias=1.0)
    lstm_out, lstm_state = tf.nn.rnn(lstm_cell, H_split, initial_state = init_cell_state)
    out = tf.add(tf.matmul(lstm_out[-1], W_['out']), B_['out'])
    return {
        'X_' : X_, 'H_' : H_, 'H_split' : H_split,\
        'lstm_out' : lstm_out, 'lstm_state' : lstm_state, \
        'out': out
    }


x = tf.placeholder("float", [None, rstep, input_dimen])
y = tf.placeholder("float", [None, output_dimen])
init_state = tf.placeholder("float", [None, hidden_dimen * 2])

rnn_res = SimpleRNN(init_state, rstep, x, WS,BS)
out = rnn_res['out']

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(out, y))
optm = tf.train.AdamOptimizer(learning_rate).minimize(cost)
accr   = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(out,1), tf.argmax(y,1)), tf.float32))
init   = tf.initialize_all_variables()

print ("Graph build")    

Graph build


In [5]:
# Training Graph
sess = tf.Session()
sess.run(init)

for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(train_img.shape[0]/batch_size)
    for i in range(num_batch):
        randidx = np.random.randint(train_img.shape[0], size=batch_size)
        batch_xs = np.reshape(train_img[randidx, :], (batch_size, rstep, input_dimen))
        batch_ys = train_label[randidx, :]                
        batch_init_state = np.zeros((batch_size, hidden_dimen * 2))
        
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys, init_state : batch_init_state})
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys,  init_state : batch_init_state})/num_batch
            
    if epoch % display_step == 0:
        train_img_acc = sess.run(accr , ({x: batch_xs, y: batch_ys, init_state : batch_init_state}))
        print ("epoch: %03d/%03d  ,  cost: %.6f  ,  train_img_acc: %.3f" \
               % (epoch, training_epochs, avg_cost, train_img_acc))

avg_acc = 0.
test_batch_size = 10
num_batch_test = int(test_img.shape[0]/test_batch_size)
for i in range(num_batch_test):
    batch_xs_test = np.reshape(test_img[i * test_batch_size : (i+1) * test_batch_size ], (test_batch_size, rstep, input_dimen))
    batch_ys_test = test_label[i * test_batch_size : (i+1) * test_batch_size ] 
    batch_init_state = np.zeros((test_batch_size, hidden_dimen * 2))
    avg_acc += sess.run(accr, feed_dict={x : batch_xs_test, y : batch_ys_test, init_state : batch_init_state})
    
print ("Training complete, Accuracy : %.6f" \
       % (avg_acc / num_batch_test,))

epoch: 000/050  ,  cost: 0.479340  ,  train_img_acc: 0.940
epoch: 010/050  ,  cost: 0.020915  ,  train_img_acc: 1.000
epoch: 020/050  ,  cost: 0.007673  ,  train_img_acc: 1.000
epoch: 030/050  ,  cost: 0.003174  ,  train_img_acc: 1.000
epoch: 040/050  ,  cost: 0.002135  ,  train_img_acc: 1.000
Training complete, Accuracy : 0.982800
